In [1]:
import requests
import json
import pandas as pd
import csv
from pandas.io.json import json_normalize
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, MetaData, Table, Column, Integer, Text, Float
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.ext.declarative import declarative_base

# youtube vid for help https://youtu.be/LYh8ih2X5Oo

In [2]:
#define function to use later, dynamically breaks down dictionary, list or nested dictionaries
g = []
def print_dict(v,prefix=''):
    if isinstance(v,dict):
        for k,v2 in v.items():
            p2="{}.{}".format(prefix,k)
            print_dict(v2,p2)
    elif isinstance(v,list):
        for i,v2 in enumerate(v):
            p2="{}.{}".format(prefix,i)
            print_dict(v2,p2)
    else:
        g.append(['{}'.format(prefix),v])
    return g

In [3]:
stub_url = "https://pogoapi.net"
endpoints = {
    "pokemon_names": "/api/v1/pokemon_names.json",
    "pokemon_types": "/api/v1/pokemon_types.json",
    "pokemon_stats": "/api/v1/pokemon_stats.json",
    "fast_moves": "/api/v1/fast_moves.json",
    "charged_moves": "/api/v1/charged_moves.json",
    "current_pokemon_moves": "/api/v1/current_pokemon_moves.json",
}

url_list = []
for key in endpoints:
    url_list.append(stub_url + endpoints[key])

In [4]:
url_response0 = requests.get(f"{url_list[0]}").json() #first endpoint, pokemon_names

df_0=pd.DataFrame.from_dict(print_dict(url_response0),orient='columns').T #convert from dict to df then transpose
df_0.columns=df_0.iloc[0] #make first row column headers
df_0.drop(df_0.index[0], inplace=True) #drop index, perform in place to avoid another variable

names_ = []  #create empty list to hold column names
for i in df_0.columns:
    names_.append(i.split('.')[2]) #split column names on decimal delimiter
df_0.columns=names_    #set new column names as headers
df_id = df_0['id'].T.rename(columns={1:"id"}).reset_index(drop=True)
df_name = df_0['name'].T.rename(columns={1:"name"}).reset_index(drop=True)
frames = [df_id, df_name]
pokemon_names = pd.concat(frames,axis=1) #merge indiv columns back together
pokemon_names.head()

,id,name
0,1,Bulbasaur
1,2,Ivysaur
2,3,Venusaur
3,4,Charmander
4,5,Charmeleon


In [5]:
type(pokemon_names)

pandas.core.frame.DataFrame

In [6]:
# Create an engine for the `pokemonGo.sqlite` database
engine = create_engine("sqlite:///Resources/pokemonGo.sqlite", echo=False)
conn = engine.connect()

In [7]:
Base = declarative_base()

In [8]:
class tbl_pokemon_names(Base):
    __tablename__ = 'tbl_pokemon_names'
    
    id = Column(Integer, primary_key=True)
    name = Column(Text)

In [9]:
Base.metadata.create_all(engine)
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
table_data = pokemon_names.to_dict(orient='records')

In [11]:
pokemon_names_table = sqlalchemy.Table('tbl_pokemon_names', metadata, PrimaryKeyConstraint('id'),
                                      autoload=True, extend_existing=True)

In [12]:
conn.execute(pokemon_names_table.delete())
conn.execute(pokemon_names_table.insert(),table_data)

In [13]:
 # Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['tbl_pokemon_names']

In [14]:
NameTable = Base.classes.tbl_pokemon_names

In [15]:
session = Session(engine)

In [16]:
list_of_names = session.query(NameTable.id, NameTable.name).limit(5).all()
list_of_names

[(1, 'Bulbasaur'),
 (2, 'Ivysaur'),
 (3, 'Venusaur'),
 (4, 'Charmander'),
 (5, 'Charmeleon')]

In [17]:
session.close()